In [2]:
import pandas as pd
import numpy as np

In [3]:
src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/features/'

train_df =  pd.read_csv(src + 'df_train_spacylemmat_fullclean.csv').iloc[:, :-1]
train_df = train_df.fillna('NULL')

train_df = train_df.iloc[0:200000]
train_y = np.asarray(train_df['is_duplicate'].values)


# optional encode all train and test requires 32gb memory
# some 700 qid are repetitions which dissapears
# also there is a chance of infering absolute knowledge in test set

# # encode questions
# lbl = preprocessing.LabelEncoder()
# f1 = 'question1'
# f2 = 'question2'
# lbl.fit(list(train_df[f1].values) + list(train_df[f2].values) + list(test_df[f1].values) + list(test_df[f2].values))
# train_df[f1+'b'] = lbl.transform(list(train_df[f1].values))
# test_df[f1+'b'] = lbl.transform(list(test_df[f1].values))
# train_df[f2+'b'] = lbl.transform(list(train_df[f2].values))
# test_df[f2+'b'] = lbl.transform(list(test_df[f2].values))

# Calculate all inferences from each positive question relation pair
pos = [i for i, x in enumerate(train_y) if x == 1]

import datetime
from time import time
begintime = time()
tmp = np.asarray(range(len(train_df)))
qid1 = 'qid1'
qid2 = 'qid2'

values = train_df[qid1].values
a_pos1 = train_df.loc[pos, qid1].apply(lambda x: tmp[values==x])
ca_new1 = a_pos1.apply(lambda x: train_df.loc[x, qid2].values)
print(str(datetime.timedelta(seconds=time() - begintime)))
values = train_df[qid2].values
a_pos2 = train_df.loc[pos, qid1].apply(lambda x: tmp[values==x])
ca_new2 = a_pos2.apply(lambda x: train_df.loc[x, qid1].values)
print(str(datetime.timedelta(seconds=time() - begintime)))

values = train_df[qid1].values
b_pos1 = train_df.loc[pos, qid2].apply(lambda x: tmp[values==x])
cb_new1 = b_pos1.apply(lambda x: train_df.loc[x, qid2].values)
print(str(datetime.timedelta(seconds=time() - begintime)))
values = train_df[qid2].values
b_pos2 = train_df.loc[pos, qid2].apply(lambda x: tmp[values==x])
cb_new2 = b_pos2.apply(lambda x: train_df.loc[x, qid1].values)
print(str(datetime.timedelta(seconds=time() - begintime)))

ca_new = pd.DataFrame({'n1':ca_new1, 'n2':ca_new2})
ca_new.index = range(len(ca_new))
cb_new = pd.DataFrame({'n1':cb_new1, 'n2':cb_new2})
cb_new.index = range(len(cb_new))



0:00:38.418714
0:01:12.054010
0:01:44.889810
0:02:22.261557


In [4]:
empA = []
empB = []
empResp = []
empQA = []
empQB = []
for x in range(len(ca_new)):
    tmpA = np.append(ca_new.loc[x, 'n1'], ca_new.loc[x, 'n2'])
    tmpB = np.append(cb_new.loc[x, 'n1'], cb_new.loc[x, 'n2'])
    b = train_df.loc[pos[x], qid2]
    a = train_df.loc[pos[x], qid1]
    qb = train_df.loc[pos[x], 'question2']
    qa = train_df.loc[pos[x], 'question1']

    uniqB = [i for i, l in enumerate(tmpA) if l not in np.append(tmpB, b)]
    uniqA = [i for i, l in enumerate(tmpB) if l not in np.append(tmpA, a)]
    empA.extend([b]*len(uniqB)+[a]*len(uniqA))
    empB.extend(np.append(tmpA[uniqB],tmpB[uniqA]))
    empResp.extend(train_df.loc[np.append(a_pos1.iloc[x], a_pos2.iloc[x])[uniqB], 'is_duplicate'].values)
    empResp.extend(train_df.loc[np.append(b_pos1.iloc[x], b_pos2.iloc[x])[uniqA], 'is_duplicate'].values)
    empQA.extend([qb]*len(uniqB)+[qa]*len(uniqA))
    empQB.extend(train_df.loc[np.append(a_pos1.iloc[x], a_pos2.iloc[x])[uniqB], 'question1'].values)
    empQB.extend(train_df.loc[np.append(b_pos1.iloc[x], b_pos2.iloc[x])[uniqA], 'question2'].values)

print(str(datetime.timedelta(seconds=time() - begintime)))

empQA = np.asarray(empQA)
empQB = np.asarray(empQB)

df = pd.DataFrame({qid1: empA, qid2: empB})
sort = df.apply(lambda x: np.argsort(x)[0], axis=1)
s = [i for i, x in enumerate(sort) if x == True]
eQA = empQA.copy()
eQB = empQB.copy()
eQA[s] = empQB[s]
eQB[s] = empQA[s]
df = df.apply(lambda x: np.sort(x), axis=1)
df['question1'] = eQA
df['question2'] = eQB
df['is_duplicate'] = empResp

0:04:42.875159


In [5]:
df

,qid1,qid2,question1,question2,is_duplicate
0,25,114035,what can make physics easy to learn,how can -PRON- make physics easy to learn,1
1,27,50277,what be -PRON- first sexual experience like,what be -PRON- first sexual experience,1
2,32,6937,how will a trump presidency affect the student...,what would a trump presidency mean for current...,1
3,31,93145,what would a trump presidency mean for current...,will the trump factor affect the admission of ...,1
4,31,37617,what would a trump presidency mean for current...,what would be the scenario for indian student ...,1
5,31,12544,what would a trump presidency mean for current...,how will a trump presidency affect the student...,1
6,31,16474,what would a trump presidency mean for current...,how will a trump presidency affect the student...,1
7,31,88834,what would a trump presidency mean for current...,how will a trump presidency affect the student...,1
8,31,2067,what would a trump presidency mean for current...,how will a trump presidency affect the student...,1
9,38,88660,why do people ask quora question which can be ...,why be so many quora user post question that b...,1


In [ ]:
d = df.drop_duplicates(subset=[qid1, qid2]).sort_values([qid1, qid2])
print(str(datetime.timedelta(seconds=time() - begintime)))
print(d.shape, np.mean(d['is_duplicate']))
d.to_csv("train_extended", index=False)